# LangWatch Evaluation Tracking

## Step 1: Define our LLM pipeline

Let's create a simple RAG pipeline using LangChain, guaranteeing that we can get the output and the retrieved documents used during generation.

In [1]:
import langwatch
import pandas as pd
import time

df = pd.DataFrame(
    [
        {
            "question": "What is LangWatch?",
            "answer": "LangWatch is a platform for evaluating and improving language models.",
        },
        {
            "question": "How do I use LangWatch?",
            "answer": "You can use LangWatch by installing the LangWatch SDK and then calling the LangWatch API.",
        },
        {
            "question": "Does LangWatch support multiple language models?",
            "answer": "Yes, LangWatch is compatible with all language models by using LiteLLM under the hood.",
        },
        {
            "question": "Can I visualize evaluation metrics in LangWatch?",
            "answer": "Yes, LangWatch provides dashboards for visualizing key evaluation metrics.",
        },
        {
            "question": "Is there a free tier for LangWatch?",
            "answer": "LangWatch offers a free tier with limited usage, ideal for small projects and evaluation.",
        },
        {
            "question": "Where can I find documentation for LangWatch?",
            "answer": "You can find the official documentation on the LangWatch website or GitHub repository.",
        },
    ]
)
evaluation = langwatch.evaluation.init("my-incredible-experiment")

for index, row in evaluation.loop(df.iterrows()):
    time.sleep(0.1)
    print("index", index)

Follow the results at: https://app.langwatch.ai/inbox-narrator/experiments/my-incredible-experiment?runId=proud-boisterous-mantis


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

2025-05-21 21:48:25,078 - langwatch.utils.initialization - INFO - Setting up LangWatch client...
2025-05-21 21:48:25,089 - langwatch.client - INFO - Configuring OTLP exporter with endpoint: https://app.langwatch.ai/api/otel/v1/traces
2025-05-21 21:48:25,090 - langwatch.client - INFO - Registering atexit handler to flush tracer provider on exit
2025-05-21 21:48:25,090 - langwatch.client - INFO - Successfully configured tracer provider with OTLP exporter
Setting up REST API client
2025-05-21 21:48:25,090 - langwatch.utils.initialization - INFO - LangWatch client setup complete
index 0
index 1
index 2
index 3
index 4
index 5


In [6]:
# async

import random
import time

evaluation = langwatch.evaluation.init("my-incredible-experiment")

for index, row in evaluation.loop(df.iterrows()):
    def evaluate(index, row):
        # time.sleep(random.randint(0, 10))
        print("index", index)
    evaluation.submit(evaluate, index, row)

Follow the results at: https://app.langwatch.ai/inbox-narrator/experiments/my-incredible-experiment?runId=clay-elephant-of-vigor


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

index 2
index 0
index 3
index 4
index 1
index 5


In [3]:
for index, row in df.iterrows():
    print(row.to_dict())

{'_is_copy': None, '_mgr': SingleBlockManager
Items: Index(['question', 'answer'], dtype='object')
NumpyBlock: 2 dtype: object, '_item_cache': {}, '_attrs': {}, '_flags': <Flags(allows_duplicate_labels=True)>, '_name': 0}
{'_is_copy': None, '_mgr': SingleBlockManager
Items: Index(['question', 'answer'], dtype='object')
NumpyBlock: 2 dtype: object, '_item_cache': {}, '_attrs': {}, '_flags': <Flags(allows_duplicate_labels=True)>, '_name': 1}


In [17]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="langwatch/python-sdk/.env")

from langchain.prompts import ChatPromptTemplate

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.documents import Document


loader = WebBaseLoader("https://docs.langwatch.ai")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retrieved_documents = []

# Wrap the FAISS retriever so that we can capture which documents were used to generate the response
@tool
def langwatch_search(
    query: str
) -> list[Document]:
    """"Search for information about LangWatch. For any questions about LangWatch, use this tool if you didn't already"""

    global retrieved_documents
    retrieved_documents = retriever.get_relevant_documents(query)
    return retrieved_documents

tools = [langwatch_search]
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that only reply in short tweet-like responses, use tools only once.\n\n{agent_scratchpad}",
        ),
        ("human", "{question}"),
    ]
)
agent = create_tool_calling_agent(model, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=False)  # type: ignore

output = executor.invoke({"question": "What is LangWatch?"})["output"]

print("")
print("retrieved_documents:", [ d.page_content for d in retrieved_documents])
print("output:", output)


retrieved_documents: ['Introduction - LangWatchLangWatch home pageSearch...llms.txtSupportDashboardlangwatch/langwatchlangwatch/langwatchSearch...NavigationGet StartedIntroductionDocumentationOpen DashboardGitHub RepoGet StartedIntroductionSelf HostingCookbooksLLM ObservabilityOverviewConceptsLanguage APIs & SDKsUser EventsMonitoring & AlertsCode ExamplesLLM EvaluationOffline EvaluationReal-Time EvaluationList of EvaluatorsDatasetsAnnotationsLLM DevelopmentPrompt Optimization StudioDSPy VisualizationLangWatch MCPPrompt VersioningAPI EndpointsTracesPromptsAnnotationsDatasetsSupportTroubleshooting and SupportStatus PageGet StartedIntroductionCopy pageWelcome to LangWatch, the all-in-one open-source LLMops platform.LangWatch allows you to track, monitor, guardrail and evaluate your LLMs apps for measuring quality and alert on issues.\nFor domain experts, it allows you to easily sift through conversations, see topics being discussed and annotate and score messages', 'For domain experts, i

## Step 2: Run the Batch Evaluation Experiment

Now we can use the dataset we have from LangWatch to run a batch evaluation experiment through our LLM pipeline, to see the results and tweak it for optimizations.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

from langwatch.batch_evaluation import BatchEvaluation, DatasetEntry


def callback(entry: DatasetEntry):
    output = executor.invoke({"question": entry["question"]})["output"]
    output = another_llm.invoke(output)

    return {"input": entry["question"], "output": output, "contexts": [d.page_content for d in retrieved_documents]}

# Instantiate the BatchEvaluation object
evaluation = BatchEvaluation(
    experiment="LangWatch RAG Experiment",
    dataset="dataset--rSAYL4HxQRXHSayq6c7A",
    evaluations=["jailbreak-detection", "faithfulness"],
    callback=callback,
)

# Run the evaluation
results = evaluation.run()
results.df

Starting batch evaluation...
Follow the results at: https://app.langwatch.ai/public-documentation-examples-YxS3Bf/experiments/langwatch-rag-experiment?runId=mysterious-peculiar-cricket


100%|██████████| 10/10 [00:45<00:00,  4.53s/it]

Batch evaluation done!


,question,input,output,contexts,jailbreak-detection,faithfulness
0,Can I customize the evaluation metrics in Lang...,Can I customize the evaluation metrics in Lang...,"Yes, you can customize evaluation metrics in L...",[Introduction - LangWatchLangWatch home pageSe...,True,0.000000
1,What programming languages are supported by La...,What programming languages are supported by La...,LangWatch supports Python and TypeScript for i...,[Introduction - LangWatchLangWatch home pageSe...,True,1.000000
2,How do I configure alerts in LangWatch?,How do I configure alerts in LangWatch?,Check the LangWatch documentation for configur...,[Introduction - LangWatchLangWatch home pageSe...,True,0.000000
3,Does it support langchain?,Does it support langchain?,"Yes, LangWatch supports LangChain! You can tra...",[Introduction - LangWatchLangWatch home pageSe...,True,0.666667
4,How does LangWatch help in LLMOps?,How does LangWatch help in LLMOps?,"LangWatch aids in LLMOps by tracking, monitori...",[Introduction - LangWatchLangWatch home pageSe...,True,0.888889
5,How can I visualize the traces collected by La...,How can I visualize the traces collected by La...,Check out the LangWatch documentation for guid...,[Introduction - LangWatchLangWatch home pageSe...,True,0.000000
6,Is it possible to automate evaluations with La...,Is it possible to automate evaluations with La...,"Yes, LangWatch allows for batch evaluations an...",[Introduction - LangWatchLangWatch home pageSe...,True,0.666667
7,What programming languages are supported by La...,What programming languages are supported by La...,Agent stopped due to max iterations.,[Introduction - LangWatchLangWatch home pageSe...,True,0.000000
8,How do I set up LangWatch in my python app?,How do I set up LangWatch in my python app?,Check out the [Python Integration Guide](https...,[Introduction - LangWatchLangWatch home pageSe...,True,0.000000
9,What are the best practices for using LangWatc...,What are the best practices for using LangWatc...,"1. **Integration**: Follow Python, TypeScript,...",[Introduction - LangWatchLangWatch home pageSe...,True,1.000000


: 